In [2]:
from z3 import *

## Utils

### At most/least one & exactly one

In [3]:
def at_least_one_seq(bool_vars):
    return Or(bool_vars)

def at_most_one_seq(x, name):
    # bool_vars renamed to x for simplicity
    n = len(x)
    s = [Bool(f"s_{i}_{name}") for i in range(n-1)]     # "name" is used in order to create a
                                                        # unique set of variables s for each constraint
                                                        # of the kind at_most_one_seq, so that they do not overlap 
                                                        # for different constraints!
                                                        # N.B. len(s) = n-1 != n = len (x)
    clauses = []
    clauses.append(Or(Not(x[0]), s[0]))                 # x[0] -> s[0] (s[i] modeled as: s[i] is true if the SUM UP TO index i IS 1!) i.e. x_1 -> s_1 in math notation
    for i in range(1, n-1):
        clauses.append(Or(Not(x[i]), s[i]))             # these two clauses model (x[i] v s[i-1]) -> s[i]
        clauses.append(Or(Not(s[i-1]), s[i]))
        clauses.append(Or(Not(s[i-1]), Not(x[i])))      # this one models s[i-1] -> not x[i]
    clauses.append(Or(Not(s[-1]), Not(x[-1])))          # s[n-2] -> not x[n-1]  i.e. s_(n-1) -> s_n in the mathematical notation (1-based)
    return And(clauses)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

In [4]:
## FORSE COSE A CASO, tentativo di somma

def add(a,b):
    #convert to binary with length digits=10
    digits = 16
    a_bin = [(a%(2**(i+1)) // 2**i)==1 for i in range(digits,-1,-1)] #BitVec(a,digits).as_binary_string()  #https://ericpony.github.io/z3py-tutorial/guide-examples.htm
    b_bin = [(b%(2**(i+1)) // 2**i)==1 for i in range(digits,-1,-1)] 

    d = [Bool(f"d_{k}") for k in range(digits)] #slide SAT-I
    c = [Bool(f"c_{k}") for k in range(digits)]
    c[0] = False

    for k in range(digits-1,-1,-1):
         d[k] == Or(And(a_bin[k],b_bin[k],c[k]), And(a_bin[k],Not(b_bin[k]),Not(c[k])), 
                                   And(Not(a_bin[k]),b_bin[k],Not(c[k])), And(Not(a_bin[k]),Not(b_bin[k]),c[k]))
         c[k-1] == Or(And(a_bin[k],b_bin[k]), And(a_bin[k],c[k]), And(b_bin[k],c[k]))

    #What if the result exceed the 16 digits supposed (c[0] True)
    
    return (d)

f = add(8,0)
print(f)

[d_0, d_1, d_2, d_3, d_4, d_5, d_6, d_7, d_8, d_9, d_10, d_11, d_12, d_13, d_14, d_15]


## Model 1

In [5]:
def multiple_couriers_planning_1(m, n, l, s, D):
    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars

    # r for routes
    r = [[[Bool(f"r_{k}_{j}_{i}") for k in range(n)] for j in range(n)] for i in range(m)]  
    # r_kji = 1 indicates that courier i delivers object j as its k-th delivery
    # O(m * n^2) vars

    s = Solver()


    ## CONSTRAINTS
    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        s.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))
    
    # Interesting question: see discussion forum (distance func. property)
    # -> answer: the distance is quasimetric => we can add implicit constraint "every courier has at least 1 package to deliver"
    # TODO: check if it actually speeds up execution
    



# Encodings Ideas

#### Idea 1 - Ordering only on objects

Instead of keeping r_k_j_i for the route of courier i, let's just keep an ordering matrix nxn, call it O, s.t. O_k_j = 1 iff object k is delivered as j-th object => i.e. we remove the courier assignment and we only use it as an ordering between objects

PRO: O(n^2) variables << O(m*n^2) variables
CON: have a lot of simmetries, because different matrices correspond to the same general set of routes if you swap orders for objects of 2 different couriers while keeping their relative position w.r.t. each courier route

#### Idea 2

Mantain the routes r_k_j_i but find a way to use a_ij = 0 => a whole row/col of r_?_?_i is full of 0s
i.e. use a structure that enables for more constraints on r, s.t. avoid the "random values below row K, with K = #objects delivered by courier i"